In [2]:
import pandas as pd

# Veri setinin dosya yolu. Eğer dosya aynı klasördeyse sadece adı yeterli.
# Eğer farklı bir yerde ise, tam dosya yolunu belirtmelisin.
file_path = 'IMDB Dataset.csv'

try:
    # CSV dosyasını oku
    df = pd.read_csv(file_path)

    print("Veri Seti Başarıyla Yüklendi!")
    print("\nİlk 5 satır:")
    print(df.head())

    print("\nVeri Seti Bilgileri:")
    df.info()

    print("\nDuygu Dağılımı:")
    print(df['sentiment'].value_counts())

    print("\nMetin Sütunu İçin Bazı İstatistikler (Karakter Sayısı):")
    df['text_length'] = df['review'].apply(len)
    print(df['text_length'].describe())

except FileNotFoundError:
    print(f"Hata: '{file_path}' dosyası bulunamadı. Lütfen dosya yolunu kontrol edin veya dosyanın aynı klasörde olduğundan emin olun.")
except Exception as e:
    print(f"Bir hata oluştu: {e}")

Veri Seti Başarıyla Yüklendi!

İlk 5 satır:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

Veri Seti Bilgileri:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB

Duygu Dağılımı:
sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Metin Sütunu İçin Bazı İstatistikler (Karakter Sayısı):
count    50000.000000
mean      1309.431020
std        989.728014
min         32.000000
25%        699.0

In [3]:
import re

def remove_html_tags(text):
    """Metindeki HTML etiketlerini kaldırır."""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 'review' sütununa HTML etiketlerini temizleme fonksiyonunu uygula
df['review_cleaned'] = df['review'].apply(remove_html_tags)

# Temizlenmiş ilk 5 yorumu ve orijinal yorumları karşılaştır
print("Orijinal ve Temizlenmiş Yorumların İlk 5 Satırı:")
print(df[['review', 'review_cleaned']].head())

# Değişikliğin olup olmadığını görmek için rastgele bir yorumu kontrol edebiliriz
# Örneğin, 3. yorumu alalım (indeksi 2)
print("\n-----------------------------------------------------")
print("Rastgele Bir Yorumun Orijinal ve Temizlenmiş Hali (Örnek Indeks 2):")
print(f"Orijinal: {df['review'].iloc[2]}")
print(f"Temizlenmiş: {df['review_cleaned'].iloc[2]}")
print("-----------------------------------------------------")

Orijinal ve Temizlenmiş Yorumların İlk 5 Satırı:
                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      review_cleaned  
0  One of the other reviewers has mentioned that ...  
1  A wonderful little production. The filming tec...  
2  I thought this was a wonderful way to spend ti...  
3  Basically there's a family where a little boy ...  
4  Petter Mattei's "Love in the Time of Money" is...  

-----------------------------------------------------
Rastgele Bir Yorumun Orijinal ve Temizlenmiş Hali (Örnek Indeks 2):
Orijinal: I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comed

In [4]:
import string
import re # Eğer regex kullanacaksan

# Küçük harfe çevirme
df['review_cleaned'] = df['review_cleaned'].str.lower()

# Noktalama işaretlerini kaldırma (bir yöntem)
# df['review_cleaned'] = df['review_cleaned'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Noktalama işaretlerini kaldırma (başka bir yöntem, regex ile)
df['review_cleaned'] = df['review_cleaned'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


# Temizlenmiş (küçük harfe çevrilmiş ve noktalama işaretleri kaldırılmış) ilk 5 yorumu kontrol et
print("Küçük Harf ve Noktalama Temizliği Sonrası İlk 5 Yorum:")
print(df[['review_cleaned']].head())

# Rastgele bir örnek yorumu tekrar kontrol edebilirsin
print("\n-----------------------------------------------------")
print("Rastgele Bir Yorumun Noktalama Temizliği Sonrası Hali (Örnek Indeks 2):")
print(f"Temizlenmiş (Yeni Hali): {df['review_cleaned'].iloc[2]}")
print("-----------------------------------------------------")

Küçük Harf ve Noktalama Temizliği Sonrası İlk 5 Yorum:
                                      review_cleaned
0  one of the other reviewers has mentioned that ...
1  a wonderful little production the filming tech...
2  i thought this was a wonderful way to spend ti...
3  basically theres a family where a little boy j...
4  petter matteis love in the time of money is a ...

-----------------------------------------------------
Rastgele Bir Yorumun Noktalama Temizliği Sonrası Hali (Örnek Indeks 2):
Temizlenmiş (Yeni Hali): i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy the plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer while some may be disappointed when they realize this is not match point 2 risk addiction i thought it was proof that woody allen is still fully in control of the style many of us have grown to loveth

In [5]:
from nltk.corpus import stopwords

# İngilizce durak kelimeleri yükle
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    """Metindeki durak kelimeleri kaldırır."""
    words = text.split() # Metni kelimelere ayır (tokenization)
    # Her kelimeyi kontrol et ve eğer durak kelime listesinde değilse, o kelimeyi koru
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words) # Kalan kelimeleri tekrar birleştir

# 'review_cleaned' sütununa durak kelime temizleme fonksiyonunu uygula
df['review_cleaned'] = df['review_cleaned'].apply(remove_stopwords)

# Temizlenmiş ilk 5 yorumu kontrol et
print("Durak Kelimeler Temizliği Sonrası İlk 5 Yorum:")
print(df[['review_cleaned']].head())

# Rastgele bir örnek yorumu tekrar kontrol et
print("\n-----------------------------------------------------")
print("Rastgele Bir Yorumun Durak Kelimeler Temizliği Sonrası Hali (Örnek Indeks 2):")
print(f"Temizlenmiş (Yeni Hali): {df['review_cleaned'].iloc[2]}")
print("-----------------------------------------------------")

Durak Kelimeler Temizliği Sonrası İlk 5 Yorum:
                                      review_cleaned
0  one reviewers mentioned watching 1 oz episode ...
1  wonderful little production filming technique ...
2  thought wonderful way spend time hot summer we...
3  basically theres family little boy jake thinks...
4  petter matteis love time money visually stunni...

-----------------------------------------------------
Rastgele Bir Yorumun Durak Kelimeler Temizliği Sonrası Hali (Örnek Indeks 2):
Temizlenmiş (Yeni Hali): thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed realize match point 2 risk addiction thought proof woody allen still fully control style many us grown lovethis id laughed one woodys comedies years dare say decade ive never impressed scarlet johanson managed tone sexy image jumped right average spirited y

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

print("\n--- Adım 4: Metin Temsili (TF-IDF Vectorization) ---")

# TF-IDF Vectorizer nesnesini oluştur
# max_features: En sık geçen 10.000 kelimeyi al (önemli kelimeleri sınırlamak için)
# min_df: Bir kelime en az 5 belgede geçmiyorsa göz ardı et (çok nadir kelimeleri elemek için)
# max_df: Bir kelime belgelerin %70'inden fazlasında geçiyorsa göz ardı et (çok yaygın kelimeleri elemek için)
vectorizer = TfidfVectorizer(max_features=10000, min_df=5, max_df=0.7)

# Temizlenmiş yorumları (df['review_cleaned']) TF-IDF vektörlerine dönüştür
# X artık modelin kullanacağı sayısal özellik matrisi olacak
X = vectorizer.fit_transform(df['review_cleaned'])

# Oluşan TF-IDF matrisinin boyutlarını kontrol et
# Çıktı (yorum sayısı, benzersiz ve filtrelenmiş kelime sayısı) şeklinde olacaktır.
print("TF-IDF Matrisinin Boyutları (Yorum Sayısı, Benzersiz Kelime Sayısı):", X.shape)

# İsteğe bağlı: Oluşturulan kelime dağarcığının (vocabulary) ilk birkaç elemanını görmek
# Eğer istersen bu satırı yorumdan çıkarıp kelimeleri görebilirsin.
# print("\nVectorizer Kelime Dağarcığından İlk 10 Kelime:")
# print(vectorizer.get_feature_names_out()[:10])

print("\nMetin temsil adımı tamamlandı.")


--- Adım 4: Metin Temsili (TF-IDF Vectorization) ---
TF-IDF Matrisinin Boyutları (Yorum Sayısı, Benzersiz Kelime Sayısı): (50000, 10000)

Metin temsil adımı tamamlandı.


In [7]:
from sklearn.model_selection import train_test_split

# 1. Sentiment etiketlerini sayısal formata çevir ('positive' -> 1, 'negative' -> 0)
# 'sentiment_numeric' adında yeni bir sütun oluşturuyoruz
df['sentiment_numeric'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# 2. Özellikler (X) ve Hedef Değişken (y) tanımla
# X zaten TF-IDF matrisimiz (bir önceki adımda oluşturuldu).
# y ise yeni oluşturduğumuz sayısal sentiment sütunu.
y = df['sentiment_numeric']

# 3. Veri setini %80 eğitim, %20 test olarak ayır
# random_state değerini sabit tutarak her çalıştığında aynı bölmeyi elde edebiliriz.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Bölümlenmiş setlerin boyutlarını kontrol et
print(f"X_train boyutu: {X_train.shape}")
print(f"X_test boyutu: {X_test.shape}")
print(f"y_train boyutu: {y_train.shape}")
print(f"y_test boyutu: {y_test.shape}")

X_train boyutu: (40000, 10000)
X_test boyutu: (10000, 10000)
y_train boyutu: (40000,)
y_test boyutu: (10000,)


In [9]:
from sklearn.linear_model import LogisticRegression
print("\n--- Adım 5: Modeli Seçme ve Eğitme (Logistic Regression) ---")

# Lojistik Regresyon modelini oluştur
# random_state: Sonuçların tekrarlanabilirliği için
# solver='liblinear': Küçük veri setleri için genellikle iyi çalışan bir algoritma
model = LogisticRegression(solver='liblinear', random_state=42)

# Modeli eğitim verileri üzerinde eğit
# Bu adım biraz zaman alabilir (40.000 yorum için)
print("Model eğitiliyor...")
model.fit(X_train, y_train)
print("Model eğitimi tamamlandı.")



--- Adım 5: Modeli Seçme ve Eğitme (Logistic Regression) ---
Model eğitiliyor...
Model eğitimi tamamlandı.
